# 5. 数据集合并和连接

实际数据挖掘中经常需要将不同数据源的数据整合到一个数据集中，Pandas 提供了许多函数让数据合并变得快速简单，不仅支持简单的拼接数据集，也包括类似数据库连接（join）和合并（merge）操作来处理有重叠字典的数据集。

In [ ]:
import pandas as pd
import numpy as npaczAqwcf23tye

## 5.1 NumPy 数组的合并

x、y、z均为NumPy数组，axis参数可以设置合并的方向，合并的结果仍然是ndarray。

In [ ]:
x=[[1,2,3],[11,12,13]]
y=[[4,5,6],[14,15,16]]
z=[[7,8,9],[17,18,19]]
np.concatenate([x,y,z], axis=1)

## 5.2 通过pd.concat实现简易合并

Pandas中有个pd.concat()函数与np.concatenate语法类似，但是配置参数更多，功能也更强大。

为了演示方便，先定义一个创建 DataFrame 的函数。

In [ ]:
def make_df(cols, ind):
    """一个简单的DataFrame"""
    data={c:[str(c)+str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

下面演示用 pd.concat() 函数拼接两个DataFrame：

In [ ]:
df1 = make_df('AB', [1,2])
df2 = make_df('AB', [3,4])
print(df1)
print(df2)
print(pd.concat([df1,df2]))

由此可见，默认情况下，DataFrame的合并是以扩展行的方式进行的（axis=0），通过设置坐标轴，可以以扩展列的方式合并数据集：

In [ ]:
df3 = make_df('AB', [0,1])
df4 = make_df('CD', [0,1])
print(df3); print(df4); 
print(pd.concat([df3,df4], axis=1))

此时，df3 和 df4 中数据的索引相同。思考：如果索引不同，会产生什么结果？

### 5.2.1 重复索引
pd.concat 方法与 np.concatenate 方法的最主要差异之一就是Pandas在合并时会保留索引，即使索引是重复的，比如下面的示例。然而，在实际应用中，我们并不希望出现重复的索引（违反了数据完整性）。pd.concat 方法提供了一些解决这个问题的方法。

In [ ]:
x = make_df('AB', [0,1])
y = make_df('AB', [2,3])
y.index = x.index #复制索引
print(x); print(y); 
print(pd.concat([x,y]))

<b>方法一：捕捉索引重复的错误</b>

如果要检测合并的结果中是否出现了重复索引，可以设置 verify_integrity 参数为 True，当出现重复索引时就会抛出异常。

In [ ]:
try:
    pd.concat([x,y], verify_integrity=True)
except ValueError as e:
        print("ValueError:", e)

<b>方法二：忽略索引</b>

如果索引不重要，可以在合并时通过设置ignore_index 参数为 True 来忽略它们，此时合并时会创建一个新的整数索引。

In [ ]:
print(x); print(y); 
print(pd.concat([x,y], ignore_index=True))

<b>方法三：增加多级索引</b>

另一种处理重复索引的方法是通过 keys 参数为数据源设置多级索引。合并后的结果是带有多级索引的 DataFrame。

In [ ]:
print(x); print(y); 
print(pd.concat([x,y], keys=['x','y']))

### 5.2.2 重复列
在实际应用中，经常会出现要合并的两个数据集有部分重复列的情况。pd.concat() 支持并集（类似于外连接outer join）和交集（内连接 inner join）。

In [ ]:
df5 = make_df('ABC', [1,2])
df6 = make_df('BCD', [3,4])
print(df5)
print(df6)
print(pd.concat([df5,df6])) #, sort=True

<b>默认的合并方式是对所有输入列进行并集合并（即外连接 outer join 的方式）。</b>

In [ ]:
print(pd.concat([df5,df6], join='inner'))

除此之外也可以指定合并的列名：

In [ ]:
print(pd.concat([df5, df6], join_axes=[df5.columns]))

### 5.2.3 append()方法与concat()方法比较
Series和DataFrame对象都支持append方法，用最少的代码实现数据集合并的功能。比如，df1.append(df2)的效果与pd.concat([df1,df2])一样。

In [ ]:
print(df1.append(df2))

需要注意的是，append()不直接更新原有对象的值，而是为合并后的数据创建一个新对象。因此，append()每次合并都要重新创建索引和数据缓存，所以并不是高效的解决方案，特别是需要进行多个数据集的合并时，建议先创建一个DataFrame列表，在用concat()函数一次性解决所有合并任务。

## 5.3 更复杂的数据集合并操作：merge()
在数据库操作中，经常会遇到两张表的连接，连接类型可以分为一对一、一对多和多对多，连接中可以指定连接的列，对于不是一一对应的列值，还可以指定连接方式（外连接、内连接、左连接、右连接）。Pandas也支持类似的操作，主要通过 merge() 方法实现。

### 5.3.1 一对一、一对多和多对多连接
<b>方式一：一对一连接</b>

In [ ]:
df1=pd.DataFrame({'employee':['Bob','Jack','Lisa','Sue'],
                'group':['Accounting','Engineering','Engineering','HR']})
df2=pd.DataFrame({'employee':['Lisa','Bob','Jack','Sue'],
                'hire_date':[2004, 2008,2012,2014]})
print(df1);print(df2);

In [ ]:
df3=pd.merge(df1,df2)
df3

<b>方式二：多对一连接</b>

In [ ]:
df4 = pd.DataFrame({'group':['Accounting','Engineering','HR'],
                   'supervisor':['Carly','Guido','Steve']})
print(df3);print(df4);print(pd.merge(df3, df4))
#调换 df3 和 df4 的顺序并不影响输出的数据集，除了列的位置会发生变化以外

<b>方式三：多对多连接</b>

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting','Engineering','Engineering', 'HR','HR'],
                   'skills': ['math', 'spreadsheets', 'coding', 'linux', 'spreadsheets','organization']})
print(df1); print(df5); print(pd.merge(df1, df5))

### 5.3.2 设置合并的键值
pd.merge() 合并数据集时会默认将两个输入的一个或者多个<b>共同列</b>作为键，实际中，两个输入要合并的列可能是不同名的，Pandas 提供了一系列指定键的方法。

<b>1、参数on的用法</b>

最简单的方法是直接将参数 on 设置为一个列名字符串或者包含多列名称的列表，此时，要求两个DataFrame使用共同列名。

In [ ]:
print(pd.merge(df1,df2,on='employee'))

<b>2、left_on和right_on参数</b>

如果两个表中的字段名不一样，可以用left_on和right_on参数来指定。

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jack', 'Lisa', 'Sue'], 
                   'salary': [70000, 80000, 120000, 90000]})
print(df1); print(df3); 
print(pd.merge(df1, df3, left_on='employee', right_on='name'))

这样得到的结果中存在一个多余的 name 列，可以通过 drop() 方法去掉。由于去掉的是一列，所以必须指定 axis=1，否则 DataFrame 默认指定参数为行索引且会删除行。

In [ ]:
print(pd.merge(df1, df3, left_on='employee', right_on='name').drop('name', axis=1))

<b>3、left_index和right_index参数</b>

这两个参数用来指定将行索引作为键来合并两个数据集。

比如，现将上面 df1、df2 的 employee 列作为索引，合并数据集：

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a); print(df2a); print(pd.merge(df1a, df2a, left_index=True, right_index=True))

试试不同 index 名称的情况：

In [ ]:
df1a = df1.set_index('employee')
df3a = df3.set_index('name')
print(df1a); print(df3a); print(pd.merge(df1a, df3a, left_index=True, right_index=True))

<b>对于以索引为键值合并数据的情况，可以用 join() 方法</b>：

In [ ]:
print(df1a.join(df2a))

In [ ]:
print(df1a.join(df3a))

如果想混合使用索引和列，可以结合left_index、left_on、right_index、right_on参数来实现。

### 5.3.3 数据集连接规则

当一个值只出现在一个数据列，却没有出现在另一个数据集列中时，需要考虑两个数据集的连接规则。与数据库操作类似，merge 方法也支持外连接（outer join）、内连接（inner join）、左连接（left join）和右连接（right join）。

<b>内连接返回两个列的交集，是缺省的连接方式（这与 concat() 方法不同，concat 的缺省方式是外连接）</b>。外连接返回两个列的并集，缺失值用 NaN 填充。左连接和右连接返回的结果分别只包含左列和右列。

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'], 'food':['fish', 'beans', 'bread']}, 
                   columns=['name','food'])
df7 = pd.DataFrame({'name':['Mary','Joseph'],
                   'drink':['wine','beer']},
                  columns=['name','drink'])
print(df6); print(df7); print(pd.merge(df6,df7)); 

可以用 how 参数来指定连接方式，试试 outer、left、right：

In [ ]:
print(pd.merge(df6,df7,how='outer')); 

比较 merge 方法和 concat 的区别，concat 如它名字，只会拼接数据集，不会处理重复数据，而 merge 能够实现两个数据集的整合。

In [ ]:
print(pd.concat([df6,df7]))

### 5.3.4 重复列名的处理

如果两个输入的 DataFrame 中有重名列的情况，pd.merge() 会自动在输出的数据集中为它们加上 \_x 和 \_y 后缀，除此之外，也可以通过 suffixes 参数自定义后缀名：

In [ ]:
df8 = pd.DataFrame({'name': ['Bob','Jake','Lisa','Sue'],
                   'rank':[1,2,3,4]})
df9 = pd.DataFrame({'name': ['Bob','Jake','Lisa','Sue'],
                   'rank':[4,2,1,3]})
print(df8)
print(df9)
print(pd.merge(df8, df9, on='name'))

In [ ]:
print(pd.merge(df8, df9, on='name', suffixes=['_L','_R']))

## 5.4 案例：美国各州的统计数据

In [ ]:
pop=pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs=pd.read_csv('state-abbrevs.csv')
print(pop.head()); print(areas.head()); print(abbrevs.head())

通过连接数据表的方式，计算各州的人口密度排名。

首先，用多对以合并获取人口数据集中各州缩写对应的全名，在合并时，还需要通过 how='outer' 确保数据没有丢失。

In [ ]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1)
print(merged.head())

检查数据是否有缺失：

In [ ]:
merged.isnull().any()

检查是哪些人口数据有缺失值：

In [ ]:
merged[merged['population'].isnull()]

好像都是跟波多黎各 PR 有关，检查这个州的数据是否都是空值：

In [ ]:
merged[merged['state/region']=='PR'].isnull().all()

并非所有数据都是空值，输出具体哪些波多黎各的数据不为空，发现2000年以后该州才有数据

In [ ]:
# 对筛选条件加()非常重要，没有括号会导致解析出错
merged[(merged['state/region']=='PR') & (pd.isna(merged['population'])==False)]

接下来检查哪些州的数据为空：

In [ ]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

填充这两个值：

In [ ]:
merged.loc[merged['state/region']=='PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region']=='USA', 'state'] = 'United States'
merged.isnull().any()

接下来，将面积数据合并进来：

In [ ]:
final = pd.merge(merged, areas, on='state', how='left')
print(final.head())
final.isnull().any()

面积列中还有缺失值，进一步检查：

In [ ]:
final['state'][final['area (sq. mi)'].isnull()].unique()
# 跟下面语句是等价的：
# final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

可以看出，缺失的是全国面积，这里既可以对各州面积求和得出总面积后插入，也可以直接去掉缺失值，因为和当前的分析任务（各州人口密度排名）无关。

In [ ]:
# 利用 inplace=True 对当前数据集进行操作，而不是生成一个副本
final.dropna(inplace=True)
# 注意：这会把波多黎各2000年前的数据也删掉，不过也符合任务需求
final.isnull().any()

先选择2010年的各州人口以及总人口数据。这里用query()函数进行快速筛选。

In [ ]:
data2010=final.query("year == 2010 & ages == 'total'")
data2010.head()

现在来计算人口密度并按序排列：

In [ ]:
# 将 data2010 的索引重置为 state 列，这样计算的结果中就会保留 state 列的值，而不是行号
# 注意：下面语句只能执行一次
# data2010.set_index('state', inplace=True)

density = data2010['population']/data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
print("Most dense areas:"); density.head()

In [ ]:
print("Least dense areas:"); density.tail()